In [1]:
import json
import h5py
import random
import numpy as np
import pandas as pd
import multiprocessing
import tensorflow as tf
from dask import dataframe as dd
from sklearn.utils import shuffle

train = False

## Preparing the data

In [2]:
if train:
    path = 'data/ensemble_train/'
    model_data_path = 'model_data/collabrative/ensemble_train/dl/'
    result_path = "personalization/ensemble_train/"
else:
    path = 'data/ensemble/'
    model_data_path = 'model_data/collabrative/ensemble/dl/'
    result_path = "personalization/ensemble/"

In [3]:
transactions = pd.read_pickle(path+'transactions.pkl')[["customer_id", "article_id"]]
customer_purchase_number = transactions.groupby("customer_id").size().to_frame("prod_number").reset_index()
transactions = transactions.merge(customer_purchase_number, on="customer_id", how="inner")

c_ids = transactions.customer_id.unique()
number_of_customer = len(c_ids)
customer_encoding = {str(c_id): i for i, c_id in enumerate(c_ids)}
p_ids = transactions.article_id.unique()
number_of_products = len(p_ids)
product_encoding = {int(p_id): i for i, p_id in enumerate(p_ids)}
with open(model_data_path+'customer_id_encoding.json', 'w') as fp:
    json.dump(customer_encoding, fp)

with open(model_data_path+'product_id_encoding.json', 'w') as fp:
    json.dump(product_encoding, fp)

transactions.customer_id = transactions.customer_id.map(customer_encoding)
transactions.article_id = transactions.article_id.map(product_encoding)
p_ids = list(product_encoding.values())
del c_ids, product_encoding, customer_encoding, customer_purchase_number

Train True: customer num: 684744 | prod num: 9857 
Train False: customer num: 684649 | prod num: 9828

In [ ]:
def prepare_data(transactions, p_ids):
    transactions = transactions.groupby(["customer_id"])['article_id']\
                                .apply(lambda x: list(x), meta=("article_ids",object))\
                                .reset_index().compute().drop_duplicates(subset=["customer_id"])

    transactions["hist_len"] = transactions.article_ids.apply(lambda x: 128 if len(x)>128 else len(x))
    transactions["prod_ids"] = transactions.apply(lambda x: random.sample(x.article_ids, x.hist_len), axis=1)
    transactions["not_prods"] = transactions.prod_ids.apply(lambda x: [p_id for p_id in random.sample(p_ids, 256) if p_id not in x][:128])

    return transactions[["customer_id", "prod_ids", "not_prods"]]

transactions = prepare_data(dd.from_pandas(transactions, npartitions=4), p_ids)
transactions.to_pickle(model_data_path+'train.pkl')
del p_ids

## Data Loader

In [ ]:
class Generator(tf.keras.utils.Sequence):
    def __init__(self, data, positive_sample_length, negative_sample_length, batch_size):
        self.data = data
        self.positive_sample_length = positive_sample_length
        self.negative_sample_length = negative_sample_length
        self.sample_length = positive_sample_length + negative_sample_length
        self.batch_size = batch_size
        if batch_size % self.sample_length != 0:
            raise ValueError("batch_size must be divisible by sum of positive_sample_length and negative_sample_length")


    def user_info_generator(self):
        for i, row in self.data[["customer_id","prod_ids","not_prods"]].iterrows():
            if len(row.prod_ids) >= self.positive_sample_length:
                pids = np.asarray(random.sample(row["prod_ids"], self.positive_sample_length) + random.sample(row["not_prods"], self.negative_sample_length))
            else:
                pids = np.asarray(random.sample(row["prod_ids"],1) + random.sample(row["not_prods"], self.sample_length -1))
            labels = np.asarray([1]*self.positive_sample_length + [0]*self.negative_sample_length)
            indices = np.arange(self.sample_length)
            np.random.shuffle(indices)
            yield tf.convert_to_tensor(np.asarray([pids[indices],labels[indices]], dtype=np.int32), dtype=tf.int32)

    def user_id_generator(self):
        for i, row in self.data[["customer_id"]].iterrows():
            customer_id = [int(row["customer_id"])] * self.sample_length
            yield tf.convert_to_tensor(customer_id, dtype=tf.int32)

class BatchGenerator(Generator):
    def __init__(self, data, positive_sample_length, negative_sample_length, batch_size):
        super().__init__(data, positive_sample_length, negative_sample_length, batch_size)
        self.mini_batch = int(batch_size / self.sample_length)
        self.batch = batch_size

        self.user_info_loader = tf.data.Dataset.from_generator(
        self.user_info_generator, output_types=tf.int32).batch(self.mini_batch, drop_remainder=True)
        self.user_id_loader = tf.data.Dataset.from_generator(
        self.user_id_generator, output_types=tf.int32).batch(self.mini_batch, drop_remainder=True)

    def _get_batch(self):
        for c_ids, info in zip(self.user_id_loader, self.user_info_loader):
            x = tf.stack([tf.reshape(c_ids, self.batch), tf.reshape(info[:,0,:], self.batch)], axis=1)
            y = tf.reshape(info[:,1,:], self.batch)
            yield x, y

class DataGenerator (BatchGenerator):
    def __init__(self, data, positive_sample_length, negative_sample_length, batch_size, validation=False):
        super().__init__(data, positive_sample_length, negative_sample_length, batch_size)
        self.len_data = len(self.data) // self.batch_size
        self.validation = validation
        if validation:
            self.val_data = self.data.copy()
            self.data = self.val_data.sample(frac=0.1)

    def on_epoch_end(self):
        if self.validation:
            self.data = self.val_data.sample(frac=0.1)
        else:
            pass

    def __len__(self):
        return self.len_data

    def __getitem__(self, index):
        return next(iter(self._get_batch()))

    def get(self):
        return self._get_batch()

In [ ]:
train_data = pd.read_pickle(model_data_path+"train.pkl")
train_data = dd.from_dataframe(train_data, npartitions=4)

In [ ]:
batch_size = 512
train_generator = DataGenerator(train_data, 3, 5, batch_size)

## Matrix Factorization Model (GMF)

In [3]:
class GmfNet(tf.keras.Model):
    def __init__(self, num_users, num_prods, embedding_size, **kwargs):
        super(GmfNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_prods = num_prods
        self.embedding_size = embedding_size
        self.user_embedding = tf.keras.layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer= tf.keras.regularizers.l2(1e-6),
        )
        self.user_bias = tf.keras.layers.Embedding(num_users, 1)
        self.prod_embedding = tf.keras.layers.Embedding(
            num_prods,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer= tf.keras.regularizers.l2(1e-6),
        )
        self.prod_bias = tf.keras.layers.Embedding(num_prods, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        prod_vector = self.prod_embedding(inputs[:, 1])
        prod_bias = self.prod_bias(inputs[:, 1])
        dot_user_prod = tf.tensordot(user_vector, prod_vector, 2)
        x = dot_user_prod + user_bias + prod_bias

        return tf.nn.sigmoid(x)

In [4]:
class MlpNet(tf.keras.Model):
    def __init__(self, num_users, num_prods, embedding_size, **kwargs):
        super(MlpNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_prods = num_prods
        self.embedding_size = embedding_size
        self.user_embedding = tf.keras.layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer= tf.keras.regularizers.l2(1e-6),
        )
        self.prod_embedding = tf.keras.layers.Embedding(
            num_prods,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer= tf.keras.regularizers.l2(1e-6),
        )

        self.prediction = tf.keras.Sequential([
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.Dense(embedding_size, activation="relu", name="layer1"),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(embedding_size, activation="relu", name="layer2"),
            tf.keras.layers.Dropout(0.1),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(embedding_size, activation="relu", name="layer3"),
            tf.keras.layers.Dense(embedding_size, activation="relu", name="layer4"),
        ])

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        prod_vector = self.prod_embedding(inputs[:, 1])

        return self.prediction(tf.concat([user_vector, prod_vector], axis=1))

In [5]:
class RecommenderNet(tf.keras.Model):
    def __init__(self, num_users, num_prods, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.gmf = GmfNet(num_users, num_prods, embedding_size)
        self.mlp = MlpNet(num_users, num_prods, embedding_size)
        self.pred = tf.keras.layers.Dense(1, activation="sigmoid")

    def call(self, inputs):
        gmf_output = self.gmf(inputs)
        mlp_output = self.mlp(inputs)
        return self.pred(tf.concat([gmf_output, mlp_output], axis=1))

In [ ]:
#model = RecommenderNet(number_of_customer, c, 128) #1362281 104547
model = RecommenderNet(1362281, 104547, 128)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"])
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="weihts",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
cores = multiprocessing.cpu_count()
history = model.fit(train_generator, epochs=25, callbacks=[es, checkpoint], workers=6)

## Model Prediction

In [6]:
with open(model_data_path + 'customer_id_encoding.json', 'r') as fp:
    customer_encoding = json.load(fp)

with open(model_data_path + 'product_id_encoding.json', 'r') as fp:
    product_encoding = json.load(fp)
    product_encoding = {int(k): v for k,v in product_encoding.items()}

In [7]:
customers = pd.read_pickle(path+"customers.pkl")
customers.drop_duplicates(subset=["customer_id"], inplace=True)
customers.sort_values(by="customer_id", inplace=True)
customers = customers["customer_id"].map(customer_encoding).values
articles = pd.read_pickle(path+"articles.pkl")
articles.drop_duplicates(subset=["article_id"], inplace=True)
articles = articles["article_id"].map(product_encoding).tolist()
number_of_articles = len(articles)
number_of_customer = len(customers)
batch_size = 128
articles = articles * batch_size

In [9]:
model = RecommenderNet(len(customers), len(articles) // batch_size, 128)
model.build(input_shape=(None, 2))
model.load_weights("weights/model.h5")

In [10]:
step = 50000 // batch_size
f = h5py.File(result_path+'dl.h5', 'w', libver='latest')
dset = f.create_dataset("dl", (number_of_customer, number_of_articles), dtype=np.float32, compression='gzip')

ptr=0
temp = []
loop_size = number_of_customer + batch_size
for i,batch_i in enumerate(range(batch_size, loop_size, batch_size)):
    customer_ids_batch = customers[batch_i-batch_size:batch_i]
    customer_ids_batch = np.repeat(customer_ids_batch, len(articles)/batch_size)
    inputs = np.vstack((customer_ids_batch, articles[:customer_ids_batch.shape[0]])).T
    tf_inputs = tf.convert_to_tensor(inputs, dtype=tf.int32)

    temp += model(tf_inputs).numpy().reshape(-1).tolist()
    if i != 0 and (i % step == 0 or i == (number_of_customer // batch_size)):
        temp = np.asarray(temp, dtype=np.float32).reshape(len(temp)//number_of_articles, number_of_articles)
        dset[ptr:ptr+temp.shape[0],:] = temp
        ptr = (i+1) * batch_size
        temp = []
    print('\r' + f'{i*batch_size}: %{round(100*i*batch_size/number_of_customer, 2)}', end='')
f.close()

684544: %99.98